# 🧫🦠 Modeling microbiota-wide metabolism with MICOM

This notebook will accompany the second session of the 2023 ISB Microbiome Course. The presentation slides can be [found here](https://gibbons-lab.github.io/isb_course_2023/micom).

You can save your own local copy of this notebook by using `File > Save a copy in Drive`. You may be promted to cetify the notebook is safe. We promise that it is 🤞

**Disclaimer:**
The linear and quadratic programming problems MICOM has to solve are very large and very complicated. There are some very good commercial solvers that are very expensive (even though they are often free for academic use). To make this tutorial as accessible as possible we will use the Open Source solver [OSQP](https://osqp.org/), which is installed along with MICOM. OSQP is amazing with quadratic programming problems (kudos!) but not as accurate for linear problems. Solvers usually only guarantee a solution within a certain numerical tolerance of the real solution. In order to make everything work with OSQP this tolerance has to be relaxed to about 10<sup>-3</sup>. This means that any result with an absolute value smaller than that might very well be zero so we should look at larger values only. Installing cost-free academic versions of commercial solvers like [IBM CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [Gurobi](https://www.gurobi.com/) would alow you to lower the tolerance to 10<sup>-6</sup>.



# 📝 Setup

MICOM installation is is usually pretty straight-forward and can be as easy as typing `pip install micom` into your Terminal.

First let's start by downloading the materials again and switching to the folder.

In [ ]:
!git clone https://github.com/gibbons-lab/isb_course_2023 materials
%cd materials

## Basic Installation

Installing MICOM is straight-forward in Python. OSQP itself will be installed automatically along with it.

## Enable QIIME 2 interactions

Before we start, we also need to install packages to read the "biom" file format used by QIIME 2 to save tables. This is only necessary if you want to read QIIME 2 FeatureTable artifacts (like the ones we constructed yesterday)

In [ ]:
!pip install -q micom Cython biom-format

print("Done! 🎉 ")

Okay, all done. So let's get started building some models 🦺🛠d😁.

# 💻 MICOM

We will use the Python interface to MICOM since it plays nicely with Colaboratory. However, you could run the same steps within the QIIME 2 MICOM plugin ([q2-micom](https://library.qiime2.org/plugins/q2-micom/26/)).

Here is an overview of all the steps and functions across both interfaces:
![micom overview](https://github.com/micom-dev/q2-micom/raw/706f583a060b91c12c0cec7acea2354fdd0dd320/docs/assets/overview.png)

The process of building a metabolic model in MICOM begins with constructing a combined abundance/taxonomy table, referred to hereafter as a taxonomy table. Let's load a sample taxonomy table to see what it looks like:



In [ ]:
from micom.data import test_data

test_data().head()

In this taxonomy table, we see four identical strains of _E. coli_ (1 through 4), across two samples (sample_1 and sample_2). We can see that each row represents a single taxon in a single sample, and the `abundance` column identifies the abundance of that taxon in the sample.

The `id` column specifies identifiers for the taxa and should be expressive and not include spaces or special characters. Since we are using a taxonomy database to build our models (more on that soon), we don't need a `file` column.

You might notice that this dataframe looks very different from what we generated in yesterday's tutorial, where we ended up with separate QIIME 2 artifacts 😱

No worries, we can deal with that.

## Importing data from QIIME 2

MICOM can read QIIME 2 artifacts. You don't even need to have QIIME 2 installed for that! But before we do so, let's resolve one issue. We discussed that MICOM summarizes genome-scale models into pangenome-scale models as a first step, but our data are on the ASV level...so how will we know what to summarize?

Basically, a specific model database can be used to quickly summarize pangenome-scale models for use within MICOM. So, before we read our data we have to decide which model database to use. We will go with the [AGORA database](https://pubmed.ncbi.nlm.nih.gov/27893703/), which is a curated database of more than 800 bacterial strains that commonly live in the human gut. In particular, we will use a version of this database summarized on the genus rank which can be downloaded from the [MICOM data repository](https://doi.org/10.5281/zenodo.3755182), which contains a whole lot of prebuilt databases. This database is available from the materials folder that we previously cloned.

Now we're all set to start building models! The data we previously collected can be found in the `treasure_chest` folder, so we can use those files to build our taxonomy for MICOM.

In [ ]:
from micom.taxonomy import qiime_to_micom

tax = qiime_to_micom(
    "treasure_chest/dada/table.qza",
    "treasure_chest/dada/taxa.qza",
    collapse_on="genus"
)
tax

Notice the `collapse_on` argument. That will specify the rank on which to sumarize and can be a list of several ranks. When matching taxonomy you can either match by the particular rank of interest (for example, just comparing genus names here), or you could compare the entire taxonomy, which will require all taxonomic ranks prior to the target rank to match. For that you cloud specify `collapse_on=["kingdom", "phylum", "class", "order", "family", "genus"]`.

Taxonomic names will often not match 100% between databases. For instance, the genus name "Prevotella" in one database may be "Prevotella_6" in another. The more ranks you use for matching the more likely you are to run into these issues. However, the more taxonomic ranks you use to match the more confident you can be that your observed taxon really is the same taxon as the one in the model database.

The resulting table will contain the same abundances but it will include more ranks if `collapse_on` is a list. All ranks present in the taxonomy will be used when matching to the database. We will stick with the "lax" option of only matching on genus ranks.

Let's now take a look at the taxonomy table we generated:

That looks more like the example! Again, we have a row for each taxon in each sample, so we're good to go.

One helpful thing to do is to merge in our metadata, so we'll have it at hand for the following steps. In our case, the metadata will include the ethnic group, region, and subsistence type of each of the study participants.

In [ ]:
import pandas as pd

metadata = pd.read_table("data/metadata.tsv").rename(columns={"sample-id": "sample_id"})
tax = pd.merge(tax, metadata, on="sample_id")
tax

Ok, now we want to invade our samples with C. diff. The goal is to predict susceptibility to invasion and see how disease context can influence predicted engraftment. To do this we will introduce 10% C. diff to all the samples! You can read more about this approach and its applications [here](https://www.biorxiv.org/content/10.1101/2023.04.28.538771v2)  

In [ ]:
def invade(data,invader,invader_rel): #define a function that will handle community invasion task
  invaded=pd.DataFrame()   # set up results dataframe
  for smp,df in data.groupby(by='sample_id'): # loop through data, one sample at a time
    df=df[df.relative>0.05].copy()            # filter out genera below 5% (this is a high threshold, which will make simulation run faster)
    df=df[df.genus!='Clostridioides']         # some samples already have C. diff, so lets remove it and then re-introduce
    abund=df.abundance.sum()*invader_rel/(1-invader_rel)  # calculate the abundance needed to achieve desired relative abundance

    info=df.iloc[0,:].copy()  # get necessary sample info
    info.genus=invader        # add invader name
    info.id=invader
    info.abundance=abund      # add invader abundance
    df.loc[df.shape[0]+1,info.index]=info.values  #append invader info
    df.relative=df.abundance.apply(lambda x:x/df.abundance.sum()) # re-calculate relative abundance
    invaded=pd.concat([invaded,df]) # append results to output dataframe
  return invaded

invaded=invade(tax,'Clostridioides',0.1) #lets test the function using Clostridioides introducd at 10%
invaded

With our taxonomy table ready to go, and our metadata merged, its finally time to get to the model building! 🎉

## Building community models

With the data we have now, building our models is pretty easy. We just pass our taxonomy table and model database to MICOM. We will remove all taxa that make up less than 5% of the community to keep the models small and speed up this tutorial. We will also have to specify where to write the models. For simplicity, we'll run this process in parallel over two threads. It should take around 10 minutes to finish.

In [ ]:
from micom.workflows import build
from micom import Community
import pandas as pd

manifest = build(invaded, "agora103_genus.qza", "models", solver="osqp",
                 cutoff=0.05, threads=2)

You'll see a warning pop up indicating that less than 50% of the abundances can be matched to the database for one of the samples. This can happen with some data, and may indicate the models may not be completely representative of the samples.Typically a fraction of 80% or more is considered great. We'll continue, but remember to keep an eye out for this in future projects!

In lower-biomass 16S amplicon sequencing samples from stool, many reads can match to food components or to host mitochondria and these hits probably do not contribute much to bacterial community metabolism. These hits will be excluded from MICOM.

Let's take a look what we got back from the `build` process.

In [ ]:
manifest

This will tell you how many taxa were found in the database and what fraction of the total abundance was represented by the database. For most samples, this looks okay (i.e., >70% of abundance represented).

So we now have our community models and can leverage MICOM fully by simulating community growth - let's discuss what we want to look at.

### Microbiome Context

Now that our models are ready to go, let's think about some of the insights we might gain from these samples. First and foremost, we want to investigate the invasion potential of C. diff. How does its ability to invade vary in samples from healthy donors versus individuals with dysbiotic gut microbiomes (pre-FMT)?

Additionally, we can use MICOM to take a mechanistic look at what metabolic strategies are leveraged by C. diff (e.g., what niche(s) does it occupy) in these different contexts.

All that and more, coming up. Stay tuned!

First we need to import our dietary context. For simplicity we will be using a formulation that represents an average western diet, but if information about host diet is known other formulations can be used (e.g., vegetarian or vegan diet). Additional dietary formulations can be found [here]( https://github.com/micom-dev/media/tree/main/media)

In [ ]:
from micom.qiime_formats import load_qiime_medium
medium = load_qiime_medium("western_diet_gut_agora.qza")
medium

### Growing the models
Great, now we have our media & our models, it's time to get growing. This will take some time, so we'll use that time as an opportunity to discuss more in depth what these processes do, and what to look for in the results. First, let's run the `grow()` command. This will take the models we've built, and find an optimal solution to the fluxes based upon the medium that's been applied. Should take about 10 minutes to complete.

If that takes too long or was aborted, we can read it in from the treasure chest.

In [ ]:
from micom.workflows import grow, save_results

growth = grow(manifest, "models",medium, tradeoff=0.8, threads=2)

# We'll save the results to a file
save_results(growth, "growth.zip")

Again, if that takes too long or was aborted, we can read it in from the treasure chest.

In [ ]:
from micom.workflows import load_results

try:               # Will only run if the previous step failed
  growth
except NameError:
  growth = load_results("treasure_chest/growth.zip")

What kind of results did we get? Well, `grow` returns a tuple of 3 data sets:

1. The predicted growth rate for all taxa in all samples
2. The import and export fluxes for each taxon and the external environment
3. Annotations for the fluxes mapping to other databases

### 📈 Growth Rates

The growth rates are pretty straightforward.

In [ ]:
growth_rates=pd.merge(growth.growth_rates,metadata,on='sample_id')
growth_rates

### ↔️ Exchange Fluxes

More interesting are the exchange fluxes. These reactions represent the import and export of metabolites from the system Let's look at those now:

In [ ]:
growth.exchanges

So we see how much of each metabolite is either consumed or produced by each taxon in each sample. `tolerance` denotes the accuracy of the solver and tells you the smallest absolute flux that is likely different form zero (i.e., substantial flux). *All of the fluxes are normalized to 1g dry weight of bacteria*. So, you can directly compare fluxes between taxa, even if they are present at very different abundances.

If you're curious what the abbreviation for each of these metabolites represents, that can be found in the annotations dataframe. For instance, let's find out what `"tre[e]"` represents.

In [ ]:
anns = growth.annotations
anns[anns.metabolite == "tre[e]"]

Trehalose! Interesting, [that's an important metabolite](https://pubmed.ncbi.nlm.nih.gov/34277467/) in the context of CDI! All of these annotations and more information at are also available at https://vmh.life, maintained by Dr. Ines Thiele's lab.


# 📊 Visualizations

Let's visualize our results. Because of the rich output of these models, it can be overwhelming to represent it all, but don't worry! There are tools in place for this already.

We will use the standard visualizations included in MICOM. These tools take in the growth results we obtained before and create visualizations in standalone HTML files that bundle the plots and raw data and can be viewed directly in your browser.

First, let's look at the growth rates of each taxon across samples.

In [ ]:
from micom.viz import *

viz = plot_growth(growth, filename="growthrates.html")

Normally, we could call `viz.view()` afterwards and it would open it in our web browser. However, this will not work in Colab. However, the plot function creates the file `growth_rates_[DATE].html` in your `materials` folder. To open it, simply download that file and view it in your web browser. We can see that there are many things going on, but it's not super clear. Let's continue.

We're interested in understanding the invasion potential of C. diff so lets extract the predicted C. diff growth rates. In addition to C. diff growth rate we can also look at what fraction of the community growth rate this represents.



In [ ]:
cdiff = growth_rates[growth_rates.taxon=='Clostridioides'].copy()
cdiff

Now that we've extracted the C. diff specific growth rates lets take a look at how they compare between patients with healthy and disbiotic gut microbiomes

In [ ]:
import seaborn as sns
sns.boxplot(x='disease_state',y='growth_rate',data=cdiff)

Looks like C. diff is predicted to grow in all samples but its predicted growth rate is ~2x higher in the Pre-FMT samples. You can see there is also a decent amount of variation in the Pre-FMT results.

## Growth niches

Another thing we can look at is whether individual taxa inhabit different growth niches across different disease contexts. Here we can use the `plot_exchanges_per_taxon` function to see how exchanges differ within and between taxa, within and across human populations.

In [ ]:
plot_exchanges_per_taxon(growth, perplexity=4, direction="import", filename="niche.html")


This function projects the full set of import or export fluxes onto a two dimensional plane, and arranges taxa so that more similar flux patterns lie nearer together. Taxa closer to one another compete for a more similar set of resources (and/or produce a more similar set of metabolites). The center of the plot signifies a more competitive nutrient space, whereas clusters on the outskirts denote more isolated niches.

You can tune [TSNE parameters](https://distill.pub/2016/misread-tsne/), such as perplexity, to get a more meaningful grouping. We will lower the perplexity here since we don't have a lot of data points.


One small take away from this analysis is the speration between C. diff pre- and post- FMT samples, suggesting that C. diff may leverage different metabolic strategies in these contexts. Lets take a closer look at this...

## Comparative flux analysis

Now let's compare the metabolite imports between the two disease contexts. We're interested to see how the consumption profile of the microbiome changes when the disease state changes, as changes in microbiome context can lead to changes in host succeptibility to infection.  To look into this deeper, we'll transform the microbiome import data and then plot the metabolite imports on a heatmap.

We can use the `consumption_rates` function in MICOM to calculate consumption rates from the growth results. This will tell us what the patient microbiomes are consuming and provide additional insight into available niches. To visualize the results we'll  run a centered log ratio transformation on the data, to account for the compositional nature of these data and compare all the fluxes against each other. Importantly, here we consider the consumption rates for samples with no C. diff present to understand how the initial state of the patient microbiomes may influence invasion potential.

In [ ]:
from micom.workflows import load_results
from micom.measures import consumption_rates
import numpy as np
no_cdiff_growth = load_results("treasure_chest/no-cdiff-growth.zip")  # Load growth results with no C. diff invasion
exchanges = consumption_rates(no_cdiff_growth)         # extract consumption rates
exchanges=pd.merge(exchanges,metadata,on='sample_id')  # add metadata
exchanges = pd.pivot_table(                            # convert to a matrix of samples vs. metabolites
    exchanges,                                         # that contains the production rates
    index = ['disease_state','sample_id'],
    columns = 'name',
    values = 'flux'
)
exchanges = exchanges.T.fillna(0.0)                    # if a metabolite is not produced its flux is zero
exchanges = exchanges.apply(                           # ...and a CLR transform again, normalizes the fluxes
    lambda xs: np.log(xs + 0.001) - np.log(xs.mean() + 0.001),
    axis=0)
exchanges = exchanges.reindex(                         # sort by variance, highest variance fluxes first
    exchanges.var(axis = 1).sort_values(ascending=False).index
)

exchanges

We can use seaborn to plot our heatmap:

In [ ]:
import seaborn as sns
import numpy as np

sns.clustermap(
    exchanges.head(50),  # take 50 highest fluxes
    cmap = 'viridis',
    yticklabels = True,  # show all metabolite names
    figsize = (8, 12)    # size of the heatmap
)

We can see here that the disease context is important - there are significant differences in consumption rates between the healthy and pre-fmt microbiomes. These differences may why C. diff can exploit the pre-FMT microbiomes and achieve higher predicted growth rates.

# 🏫 Exercises

Time for you to try your hand at some analysis, lets take a closer looks at the metabolic strategies used by C. diff

## Metabolic strategies used by C. diff
We've alread looked at the community wide consumption fluxes in the absencence of C. diff and found that they differ between disease contexts. What about the import fluxes of C. diff specifically? Can you develop a visualization to look at those?

In [ ]:
exchanges=#your code here                                            # extract exchanges from growth data
cdiff_exchanges=#your code here                                      # get cdiff specific exchanges
cdiff_imports=#your code here                                        # specifically look at imports
cdiff_imports=#your code here                                        # add metadata
cdiff_imports = pd.pivot_table(#your code here)                      # convert to a matrix of samples vs. metabolites
                                                                     # that contains the production rates
cdiff_imports = abs(cdiff_imports.T.fillna(0.0))                     # fill nans with 0s

annot=#your code here                                                # optionally map reactions to metabolite names
annot.index=annot.reaction
cdiff_imports.index=cdiff_imports.index.map(annot.name.to_dict())    # not necessary but makes results more human readible

cdiff_imports =  #your coder here                                    # ...and a CLR transform again, normalizes the fluxes

cdiff_imports =  #your code here                                     # sort by variance, highest variance fluxes first

cdiff_imports

In [ ]:
#Make a heatmap with top 50 fluxes with highest variance

#your code here

# 🔵 Addendum


## Choosing a tradeoff value

Even if you don't have growth rates available you can still use your data to choose a decent tradeoff value. This can be done by choosing the largest tradeoff value that still allows growth for the majority of the taxa that you observed in the sample (if they are present at an appreciable abundance, they should be able to grow). This can be done with the `tradeoff` workflow in MICOM that will run cooperative tradeoff with varying tradeoff values, which can be visualized with the `plot_tradeoff` function.

In [ ]:
manifest

In [ ]:
from micom.workflows import tradeoff
import micom

tradeoff_results = tradeoff(manifest, "models", medium, threads=2)
tradeoff_results.to_csv("tradeoff.csv", index=False)

plot_tradeoff(tradeoff_results, tolerance=1e-4)

After opeing `tradeoff_[DATE].html` you will see that, for our example here, all tradeoff values work great. This is because we modeled very few taxa, which keeps the compettion down. If you would allow for fewer abundant taxa in the models, this would change drastically. For instance, here is an example from a colorectal cancer data set:

[![tradeoff example](https://micom-dev.github.io/micom/_images/tradeoff.png)](https://micom-dev.github.io/micom/_static/tradeoff.html)

You can see how not using the cooperative tradeoff would give you nonsense results where only 10% of all observed taxa grew. A tradeoff value of 0.6-0.8 would probably be a good choice for this particular data set.